<table style="width:100%">
<tr>
<td style="vertical-align:middle; text-align:left;">
<font size="2">
<a href="http://mng.bz/orYv">『Build a Large Language Model From Scratch』</a>書籍（著者 <a href="https://sebastianraschka.com">Sebastian Raschka</a>）向けの補足コードである<br>
<br>コードリポジトリ: <a href="https://github.com/rasbt/LLMs-from-scratch">https://github.com/rasbt/LLMs-from-scratch</a>
</font>
</td>
<td style="vertical-align:middle; text-align:left;">
<a href="http://mng.bz/orYv"><img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/cover-small.webp" width="100px"></a>
</td>
</tr>
</table>

# 第5章: ラベルなしデータでの事前学習

In [1]:
from importlib.metadata import version

pkgs = ["matplotlib", 
        "numpy", 
        "tiktoken", 
        "torch",
        "tensorflow" # For OpenAI's pretrained weights
       ]
for p in pkgs:
    print(f"{p} version: {version(p)}")

matplotlib version: 3.10.1
numpy version: 2.0.2
tiktoken version: 0.9.0
torch version: 2.6.0
tensorflow version: 2.18.0


- 本章では、基本的なモデル評価に関するコードや学習ループを実装し、LLMを事前学習させる方法を示すである
- 本章の最後では、OpenAIから公開されている事前学習済みの重みをモデルに読み込む方法も示すである

<img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/ch05_compressed/chapter-overview.webp" width=500px>

- 本章で扱うトピックは下図のとおりである

<img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/ch05_compressed/mental-model--0.webp" width=400px>

## 5.1 生成的テキストモデルの評価

- まずは、前章のコードを使ってGPTモデルを初期化する手順を簡単におさらいするである
- 次に、LLMに対する基本的な評価指標について議論するである
- そして最後に、これらの評価指標を学習用および検証用データセットに適用するである

### 5.1.1 GPTを用いたテキスト生成

- 前章のコードを用いてGPTモデルを初期化するである

In [2]:
import torch
from previous_chapters import GPTModel
# If the `previous_chapters.py` file is not available locally,
# you can import it from the `llms-from-scratch` PyPI package.
# For details, see: https://github.com/rasbt/LLMs-from-scratch/tree/main/pkg
# E.g.,
# from llms_from_scratch.ch04 import GPTModel

GPT_CONFIG_124M = {
    "vocab_size": 50257,   # Vocabulary size
    "context_length": 256, # Shortened context length (orig: 1024)
    "emb_dim": 768,        # Embedding dimension
    "n_heads": 12,         # Number of attention heads
    "n_layers": 12,        # Number of layers
    "drop_rate": 0.1,      # Dropout rate
    "qkv_bias": False      # Query-key-value bias
}

torch.manual_seed(123)
model = GPTModel(GPT_CONFIG_124M)
model.eval();  # Disable dropout during inference

- 上ではドロップアウトに0.1を設定しているが、近年のLLMの学習ではドロップアウトなしにするケースも比較的多いである
- また、最新のLLMではクエリ、キー、バリュー行列を作る`nn.Linear`層でバイアスを使用しないことも一般的であり（初期のGPTモデルとは異なる）、`"qkv_bias": False`がそれに対応しているである
- `context_length`は計算資源を節約するために256トークンに減らしている（本来のGPT-2 124Mパラメータ版では1024トークン）
  - これは多くの読者がノートPCでもコードを実行できるように配慮しているである
  - ただし、`context_length`を1024に戻すことも自由である（その場合、コードの変更は不要である）
  - また本章の最後では、`context_length`が1024のモデルを事前学習済みの重みから読み込む方法も示すである

- 次に、前章で定義した`generate_text_simple`関数を用いてテキストを生成するである
- さらに、トークンIDとテキストを相互変換するための便利な関数`text_to_token_ids`と`token_ids_to_text`を定義し、本章全体で使用するである

<img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/ch05_compressed/gpt-process.webp" width=500px>

In [3]:
import tiktoken
from previous_chapters import generate_text_simple

# Alternatively:
# from llms_from_scratch.ch04 import generate_text_simple

def text_to_token_ids(text, tokenizer):
    encoded = tokenizer.encode(text, allowed_special={'<|endoftext|>'})
    encoded_tensor = torch.tensor(encoded).unsqueeze(0) # add batch dimension
    return encoded_tensor

def token_ids_to_text(token_ids, tokenizer):
    flat = token_ids.squeeze(0) # remove batch dimension
    return tokenizer.decode(flat.tolist())

start_context = "Every effort moves you"
tokenizer = tiktoken.get_encoding("gpt2")

token_ids = generate_text_simple(
    model=model,
    idx=text_to_token_ids(start_context, tokenizer),
    max_new_tokens=10,
    context_size=GPT_CONFIG_124M["context_length"]
)

print("Output text:\n", token_ids_to_text(token_ids, tokenizer))

Output text:
 Every effort moves you rentingetic wasnم refres RexMeCHicular stren


- 上記の通り、モデルはまだ学習されていないため、まともな文章は出力されないである
- では、どのようにしてモデルが出力した文章の「良さ」を数値的に把握し、学習の進捗を追跡できるようにするか
- 次節では、そのために用いる生成出力に対する損失（ロス）を測る指標について説明するである
- また、次章で扱うLLMのファインチューニングでは、ここで示す以外の方法でモデル品質を測る方法も示すである

<br>

### 5.1.2 テキスト生成ロスの計算: クロスエントロピーとパープレキシティ

- 2つの学習例（行）を含むトークンIDの`inputs`テンソルがあるとする
- それに対応する目的のトークンIDを`targets`とし、モデルに生成してほしいトークン列を保持している
- 先述の通り、`targets`は`inputs`を1トークン分シフトしたものである（これは第2章でデータローダを実装した際に説明した）

In [4]:
inputs = torch.tensor([[16833, 3626, 6100],   # ["every effort moves",
                       [40,    1107, 588]])   #  "I really like"]

targets = torch.tensor([[3626, 6100, 345  ],  # [" effort moves you",
                        [1107,  588, 11311]]) #  " really like chocolate"]

- `inputs`をモデルに与えると、2つの入力例それぞれについて3トークンぶんのロジット（vocab_size=50,257次元）を得る
- ソフトマックスを適用すると、同じ次元のテンソルとして確率スコアを得ることができるである

In [5]:
with torch.no_grad():
    logits = model(inputs)

probas = torch.softmax(logits, dim=-1) # Probability of each token in vocabulary
print(probas.shape) # Shape: (batch_size, num_tokens, vocab_size)

torch.Size([2, 3, 50257])


- 下図は非常に小さい語彙サイズで例示しているが、前章末尾で議論したように、この確率スコアからテキストを復元する流れを図示したものである

<img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/ch05_compressed/proba-to-text.webp" width=500px>

- 前章で述べたように、ソフトマックスで得られた確率スコアに`argmax`を適用すれば予測トークンIDを得られるである
- これは50,257次元の確率スコアの中で最も値が大きい要素の位置が予測トークンIDを表す

- 2つの入力バッチに対して3トークン分あるので、2×3の予測トークンIDを得ることになるである:

In [6]:
token_ids = torch.argmax(probas, dim=-1, keepdim=True)
print("Token IDs:\n", token_ids)

Token IDs:
 tensor([[[16657],
         [  339],
         [42826]],

        [[49906],
         [29669],
         [41751]]])


- これらのトークンをデコードしてみると、`targets`とは全く異なるものになっているのがわかる（まだ学習前なので当然である）:

In [7]:
print(f"Targets batch 1: {token_ids_to_text(targets[0], tokenizer)}")
print(f"Outputs batch 1: {token_ids_to_text(token_ids[0].flatten(), tokenizer)}")

Targets batch 1:  effort moves you
Outputs batch 1:  Armed heNetflix


- これはまだ学習が行われていないためである
- モデルの重みを最適化するには、予測結果が正解（targets）からどれだけ離れているかを測る必要がある

<img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/ch05_compressed/proba-index.webp" width=500px>

- 下記のように、ターゲットに対応するトークンの確率だけを取り出すことができるである:

In [8]:
text_idx = 0
target_probas_1 = probas[text_idx, [0, 1, 2], targets[text_idx]]
print("Text 1:", target_probas_1)

text_idx = 1
target_probas_2 = probas[text_idx, [0, 1, 2], targets[text_idx]]
print("Text 2:", target_probas_2)

Text 1: tensor([7.4541e-05, 3.1061e-05, 1.1563e-05])
Text 2: tensor([1.0337e-05, 5.6776e-05, 4.7559e-06])


- これらの値をできるだけ1に近づけたい（最大化したい）
- ただし、数学的な最適化の観点からは、確率スコアそのものよりも対数（log）を最大化したほうが扱いやすい
- 詳細は本書の範囲外だが、より深く知りたい場合は、私が録画した講義[L8.2 Logistic Regression Loss Function](https://www.youtube.com/watch?v=GxJe0DZvydM)を参照してほしい

In [9]:
# Compute logarithm of all token probabilities
log_probas = torch.log(torch.cat((target_probas_1, target_probas_2)))
print(log_probas)

tensor([ -9.5042, -10.3796, -11.3677, -11.4798,  -9.7764, -12.2561])


- 次に、これらの平均値をとってみる:

In [10]:
# Calculate the average probability for each token
avg_log_probas = torch.mean(log_probas)
print(avg_log_probas)

tensor(-10.7940)


- これはできるだけ0に近づけたい値であり、現状はまだまだ小さい（マイナスが大きい）
- 実際の学習では、この対数確率の平均値を最大化するのではなく、**負**にした平均値を**最小化**する形にするのが深層学習の慣例である
- -10.7722を大きくしようとする（0に近づける）代わりに、10.7722を小さくしようとする（0に近づける）わけである
- 負の平均対数確率の値は、深層学習においてはクロスエントロピー損失とも呼ばれる

In [11]:
neg_avg_log_probas = avg_log_probas * -1
print(neg_avg_log_probas)

tensor(10.7940)


- PyTorchにはすでに`cross_entropy`という関数が実装されており、上記のステップをまとめて自動で実行してくれる

<img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/ch05_compressed/cross-entropy.webp?123" width=400px>

- まず`logits`と`targets`の形状を確認しておく:

In [12]:
# Logits have shape (batch_size, num_tokens, vocab_size)
print("Logits shape:", logits.shape)

# Targets have shape (batch_size, num_tokens)
print("Targets shape:", targets.shape)

Logits shape: torch.Size([2, 3, 50257])
Targets shape: torch.Size([2, 3])


- PyTorchの`cross_entropy`を使うには、これらのテンソルをバッチ次元でフラット化する必要がある:

In [13]:
logits_flat = logits.flatten(0, 1)
targets_flat = targets.flatten()

print("Flattened logits:", logits_flat.shape)
print("Flattened targets:", targets_flat.shape)

Flattened logits: torch.Size([6, 50257])
Flattened targets: torch.Size([6])


- `targets`はトークンID（索引）なので、ロジットの中の対応する位置を最大化したいわけである
- `cross_entropy`はソフトマックスと対数の計算も内部でまとめて行ってくれる

In [14]:
loss = torch.nn.functional.cross_entropy(logits_flat, targets_flat)
print(loss)

tensor(10.7940)


- クロスエントロピーと関連する概念としてパープレキシティがある
- パープレキシティはクロスエントロピーを指数変換したものに相当する

In [15]:
perplexity = torch.exp(loss)
print(perplexity)

tensor(48725.8203)


- パープレキシティは、各ステップでモデルが不確かな有効語彙サイズとして理解することもできる（例では48,725程度）
- より具体的には、モデルが実際の単語分布とどの程度合っているかを示す指標で、値が小さいほど良い予測をしているといえる

### 5.1.3 学習セットと検証セットのロスを計算する

- 以下では、非常に小規模なデータセット（1つの短編ストーリーのみ）でLLMを学習させる
- 理由は以下の通りである:
  - ノートPCなどGPU環境が必須でない場面でも実行しやすい
  - 学習が（数分で）すぐ終わるため、教育目的に適している
  - 公共のドメインにある文章を使用することで、著作権上の問題がなく、かつリポジトリ容量を大きくしすぎることを避けられる

- 参考までに、Llama 2 7Bは2兆トークンをA100 GPUで184,320 GPU時間かけて学習されたとされる
  - この時点でのAWSの8xA100サーバは1時間あたりおよそ30ドルほどかかる
  - ざっくり計算すると、(184,320 / 8) × 30ドル = 690,000ドル弱になる

- 以下では第2章で使用したのと同じデータセットを使用するである

In [16]:
import os
import urllib.request

file_path = "the-verdict.txt"
url = "https://raw.githubusercontent.com/rasbt/LLMs-from-scratch/main/ch02/01_main-chapter-code/the-verdict.txt"

if not os.path.exists(file_path):
    with urllib.request.urlopen(url) as response:
        text_data = response.read().decode('utf-8')
    with open(file_path, "w", encoding="utf-8") as file:
        file.write(text_data)
else:
    with open(file_path, "r", encoding="utf-8") as file:
        text_data = file.read()

- テキストが正しく読み込まれたか、先頭と末尾の100文字ほどを確認してみるである

In [17]:
# First 100 characters
print(text_data[:99])

I HAD always thought Jack Gisburn rather a cheap genius--though a good fellow enough--so it was no 


In [18]:
# Last 100 characters
print(text_data[-99:])

it for me! The Strouds stand alone, and happen once--but there's no exterminating our kind of art."


In [19]:
total_characters = len(text_data)
total_tokens = len(tokenizer.encode(text_data))

print("Characters:", total_characters)
print("Tokens:", total_tokens)

Characters: 20479
Tokens: 5145


- 5,145トークンしかないので、LLMの学習としては非常に小さいが、本書における教育用の例なので問題ないである
- 本章の最後では事前学習済みの重みをロードする例も示す

- 次に、学習セットと検証セットに分割し、第2章で実装したデータローダを使って学習バッチを作成するである
- 下図は`max_length=6`の例を示しており、実際に学習する際はLLMがサポートする`context_length`でバッチングされる
- 図中では入力トークンのみを示しているが、LLMでは次のトークンを予測するため、学習時はターゲットはこれを1個シフトした形になる

<img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/ch05_compressed/batching.webp" width=500px>

In [20]:
from previous_chapters import create_dataloader_v1
# Alternatively:
# from llms_from_scratch.ch02 import create_dataloader_v1

# Train/validation ratio
train_ratio = 0.90
split_idx = int(train_ratio * len(text_data))
train_data = text_data[:split_idx]
val_data = text_data[split_idx:]


torch.manual_seed(123)

train_loader = create_dataloader_v1(
    train_data,
    batch_size=2,
    max_length=GPT_CONFIG_124M["context_length"],
    stride=GPT_CONFIG_124M["context_length"],
    drop_last=True,
    shuffle=True,
    num_workers=0
)

val_loader = create_dataloader_v1(
    val_data,
    batch_size=2,
    max_length=GPT_CONFIG_124M["context_length"],
    stride=GPT_CONFIG_124M["context_length"],
    drop_last=False,
    shuffle=False,
    num_workers=0
)

In [21]:
# Sanity check

if total_tokens * (train_ratio) < GPT_CONFIG_124M["context_length"]:
    print("Not enough tokens for the training loader. "
          "Try to lower the `GPT_CONFIG_124M['context_length']` or "
          "increase the `training_ratio`")

if total_tokens * (1-train_ratio) < GPT_CONFIG_124M["context_length"]:
    print("Not enough tokens for the validation loader. "
          "Try to lower the `GPT_CONFIG_124M['context_length']` or "
          "decrease the `training_ratio`")

- 学習用データセットが非常に小さいのでバッチサイズも小さめにしている（バッチサイズが2）
- 参考として、Llama 2 7Bの学習時のバッチサイズは1024だったとされる

- オプションとして、データが正しく読み込まれているかの確認:

In [22]:
print("Train loader:")
for x, y in train_loader:
    print(x.shape, y.shape)

print("\nValidation loader:")
for x, y in val_loader:
    print(x.shape, y.shape)

Train loader:
torch.Size([2, 256]) torch.Size([2, 256])
torch.Size([2, 256]) torch.Size([2, 256])
torch.Size([2, 256]) torch.Size([2, 256])
torch.Size([2, 256]) torch.Size([2, 256])
torch.Size([2, 256]) torch.Size([2, 256])
torch.Size([2, 256]) torch.Size([2, 256])
torch.Size([2, 256]) torch.Size([2, 256])
torch.Size([2, 256]) torch.Size([2, 256])
torch.Size([2, 256]) torch.Size([2, 256])

Validation loader:
torch.Size([2, 256]) torch.Size([2, 256])


- さらにオプションとして、トークン数の整合性チェック:

In [23]:
train_tokens = 0
for input_batch, target_batch in train_loader:
    train_tokens += input_batch.numel()

val_tokens = 0
for input_batch, target_batch in val_loader:
    val_tokens += input_batch.numel()

print("Training tokens:", train_tokens)
print("Validation tokens:", val_tokens)
print("All tokens:", train_tokens + val_tokens)

Training tokens: 4608
Validation tokens: 512
All tokens: 5120


- 以上の準備ができたら、次のようにあるバッチのクロスエントロピー損失を計算するユーティリティ関数を定義するである
- また、データローダ内の指定数のバッチに対してまとめてロスを計算する2つめのユーティリティ関数も定義する

In [24]:
def calc_loss_batch(input_batch, target_batch, model, device):
    input_batch, target_batch = input_batch.to(device), target_batch.to(device)
    logits = model(input_batch)
    loss = torch.nn.functional.cross_entropy(logits.flatten(0, 1), target_batch.flatten())
    return loss


def calc_loss_loader(data_loader, model, device, num_batches=None):
    total_loss = 0.
    if len(data_loader) == 0:
        return float("nan")
    elif num_batches is None:
        num_batches = len(data_loader)
    else:
        # Reduce the number of batches to match the total number of batches in the data loader
        # if num_batches exceeds the number of batches in the data loader
        num_batches = min(num_batches, len(data_loader))
    for i, (input_batch, target_batch) in enumerate(data_loader):
        if i < num_batches:
            loss = calc_loss_batch(input_batch, target_batch, model, device)
            total_loss += loss.item()
        else:
            break
    return total_loss / num_batches

- CUDA対応GPUを持つマシンであれば、特に変更を加えなくてもGPU上で学習を行える
- `device`で指定したデバイスに、ミニバッチを載せるコードを書いているのもそのためである

In [25]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Note:
# Uncommenting the following lines will allow the code to run on Apple Silicon chips, if applicable,
# which is approximately 2x faster than on an Apple CPU (as measured on an M3 MacBook Air).
# However, the resulting loss values may be slightly different.

#if torch.cuda.is_available():
#    device = torch.device("cuda")
#elif torch.backends.mps.is_available():
#    device = torch.device("mps")
#else:
#    device = torch.device("cpu")
#
# print(f"Using {device} device.")


model.to(device) # no assignment model = model.to(device) necessary for nn.Module classes


torch.manual_seed(123) # For reproducibility due to the shuffling in the data loader

with torch.no_grad(): # Disable gradient tracking for efficiency because we are not training, yet
    train_loss = calc_loss_loader(train_loader, model, device)
    val_loss = calc_loss_loader(val_loader, model, device)

print("Training loss:", train_loss)
print("Validation loss:", val_loss)

Training loss: 10.98758347829183
Validation loss: 10.98110580444336


<img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/ch05_compressed/mental-model-1.webp" width=400px>

## 5.2 LLMの学習

- いよいよLLM学習のコードを示すである
- ここではシンプルな学習関数を実装している（学習率ウォームアップ、コサインアニーリング、勾配クリッピングなどの高度なテクニックを組み込む例は[付録D](../../appendix-D/01_main-chapter-code)を参照）

<img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/ch05_compressed/train-steps.webp" width=300px>

In [26]:
def train_model_simple(model, train_loader, val_loader, optimizer, device, num_epochs,
                       eval_freq, eval_iter, start_context, tokenizer):
    # Initialize lists to track losses and tokens seen
    train_losses, val_losses, track_tokens_seen = [], [], []
    tokens_seen, global_step = 0, -1

    # Main training loop
    for epoch in range(num_epochs):
        model.train()  # Set model to training mode
        
        for input_batch, target_batch in train_loader:
            optimizer.zero_grad() # Reset loss gradients from previous batch iteration
            loss = calc_loss_batch(input_batch, target_batch, model, device)
            loss.backward() # Calculate loss gradients
            optimizer.step() # Update model weights using loss gradients
            tokens_seen += input_batch.numel()
            global_step += 1

            # Optional evaluation step
            if global_step % eval_freq == 0:
                train_loss, val_loss = evaluate_model(
                    model, train_loader, val_loader, device, eval_iter)
                train_losses.append(train_loss)
                val_losses.append(val_loss)
                track_tokens_seen.append(tokens_seen)
                print(f"Ep {epoch+1} (Step {global_step:06d}): "
                      f"Train loss {train_loss:.3f}, Val loss {val_loss:.3f}")

        # Print a sample text after each epoch
        generate_and_print_sample(
            model, tokenizer, device, start_context
        )

    return train_losses, val_losses, track_tokens_seen


def evaluate_model(model, train_loader, val_loader, device, eval_iter):
    model.eval()
    with torch.no_grad():
        train_loss = calc_loss_loader(train_loader, model, device, num_batches=eval_iter)
        val_loss = calc_loss_loader(val_loader, model, device, num_batches=eval_iter)
    model.train()
    return train_loss, val_loss


def generate_and_print_sample(model, tokenizer, device, start_context):
    model.eval()
    context_size = model.pos_emb.weight.shape[0]
    encoded = text_to_token_ids(start_context, tokenizer).to(device)
    with torch.no_grad():
        token_ids = generate_text_simple(
            model=model, idx=encoded,
            max_new_tokens=50, context_size=context_size
        )
    decoded_text = token_ids_to_text(token_ids, tokenizer)
    print(decoded_text.replace("\n", " "))  # Compact print format
    model.train()

- 上の学習関数を使い、以下のように実際にLLMを学習してみるである

In [27]:
# Note:
# Uncomment the following code to calculate the execution time
# import time
# start_time = time.time()

torch.manual_seed(123)
model = GPTModel(GPT_CONFIG_124M)
model.to(device)
optimizer = torch.optim.AdamW(model.parameters(), lr=0.0004, weight_decay=0.1)

num_epochs = 10
train_losses, val_losses, tokens_seen = train_model_simple(
    model, train_loader, val_loader, optimizer, device,
    num_epochs=num_epochs, eval_freq=5, eval_iter=5,
    start_context="Every effort moves you", tokenizer=tokenizer
)

# Note:
# Uncomment the following code to show the execution time
# end_time = time.time()
# execution_time_minutes = (end_time - start_time) / 60
# print(f"Training completed in {execution_time_minutes:.2f} minutes.")

Ep 1 (Step 000000): Train loss 9.781, Val loss 9.933
Ep 1 (Step 000005): Train loss 8.111, Val loss 8.339
Every effort moves you,,,,,,,,,,,,.                                     
Ep 2 (Step 000010): Train loss 6.661, Val loss 7.048
Ep 2 (Step 000015): Train loss 5.961, Val loss 6.616
Every effort moves you, and, and, and, and, and, and, and, and, and, and, and, and, and, and, and, and, and, and, and, and, and, and,, and, and,
Ep 3 (Step 000020): Train loss 5.726, Val loss 6.600
Ep 3 (Step 000025): Train loss 5.201, Val loss 6.348
Every effort moves you, and I had been.                                            
Ep 4 (Step 000030): Train loss 4.417, Val loss 6.278
Ep 4 (Step 000035): Train loss 4.069, Val loss 6.226
Every effort moves you know the                          "I he had the donkey and I had the and I had the donkey and down the room, I had
Ep 5 (Step 000040): Train loss 3.732, Val loss 6.160
Every effort moves you know it was not that the picture--I had the fact by the last